In [ ]:
# packages and modules

!pip install hazm
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade
!pip install requests nlpaug transformers

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import string
import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd
import hazm
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)
from termcolor import colored
from itertools import chain
import pickle
import nlpaug.augmenter.word as naw
import pickle
from hazm import stopwords_list

from transformers import AutoModelForMaskedLM, AutoTokenizer
from collections import Counter 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import tensorflow as tf
from termcolor import colored
import ast

from sklearn.model_selection import StratifiedKFold
import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm
from itertools import chain

from sklearn.preprocessing import LabelEncoder

In [ ]:
# the main ProsPoemParallelDataset is constructed of different files
# this file has all in one, without the duplicates

all_data = pd.read_csv('.../ProsPoemParallelDataset.csv')

In [ ]:

# cleaning phase 1-----------------------------
normalizer = hazm.Normalizer(persian_numbers=False)

# all punctuations except . and /
punct = re.sub(r'[\/\.]', '', string.punctuation) +'،؟»«…'


separators = ['خلاصه داستان:.*',
              ' حاصل کلام .*',
              'در یک کلام.*',
              ' حاصل معنی:.*',
              'خلاصه کلام:.*',
              'حاصل اینکه.*',
              'معنی بیت*',
              '[\.|؟] یعنی.*', 
              'چنانکه* .*(:)+',
              '\.( )*خلاصه.*',
              
              'به عبارتی دیگر، .*',
              '\. حاصل سخن اینکه.*',
              ' حاصل کلام: *',
              '\. بنابراین .*',
              'منظور بیت: .*',
              '\.( )*بنابراین.*',
              'حاصل مقصود اینکه.*',
              'چنانکه در آیه.*'


              ]

# extra patterns
remove_patterns = [r'\[.*\]*',  r'\(.*\)', r'-.*',
                    r'–.*', r'شرح و تفسیر غزل \S*', r'\. \. ',
                    '\. \S*( )*=',
                    '\. \S* \S*( )*=',
                    r'ص \S*', 'شرح و تفسیر بیت \S*',
                    'از این‌ها که بگذریم', 'یعنی نطفۀ موسی', 'چنانکه شاعر می‌فرماید:.*',
                    ' دنیا ترک کن یعنی', ' پیر مغان یعنی',
                    'ابیات پیشین مقدمه‌ای است برای این بیت و بیت بعد\.', 
                    ' اوضاع و احوالی که در بیت قبل گفته شد به اضافه احوال او در مصراع اول این بیت',
                    'حضرت مولانا در اینجا پاسخ می‌دهد: ', ' چنانکه گفت اند:.*',
                    'آن مار زهرآگین او را نیش زد و .*', 
                    'چه هر روز عرفای کاملی .*', 
                    'چنانکه عمر خیام می‌فرماید:.*', 
                    '\*.*',
                    '_ مولانا در این بخش جلیل.*',
                    '.* دلیل مولانا در رد ادلۀ متکلمان بدین قرار است: ',
                    'احرام = آهنگ حج کردن .*',
                    '\. تمثیل:', 
                    'این بیداری که مورد نکوهش مولاناست .*',
                    'و خلاصه همه حواس منقطع می شود',
                    'طبرسی گوید:.*', 'این سیاهکاران.*',
                    'مثلا شیر پشمین ساختگی را برای گدائی بکار می‌برند، یعنی',
                    ' خلاصۀ داستان:.*',
                    'آن دوست گفت: همراه.*',
                    '.* تشبیه شده است و می‌گوید:',
                    'یکی از اعیان ناس نقل می‌کرد:.*',
                    'نام او صد و سی بار در قرآن کریم.*', 'در سال ۱۶۵ هجری در .*', 
                    ' ابن الوقت در مثنوی به دو معنی آمده.*', 
                    'حسام الدین ادامه داد:', ' دجال به معنی دروغگوی.*',
                    ' و گاه به معنی.*', 
                    ' اکبر آبادی معتقد است.*', 
                    'بعضی از مریدان گفتند که.*', 
                    '.* حاصل این که:',
                    '.*حاصل بیت:',
                    'از نظم و انتظام خارج شده یعنی ',
                    '.* پس با این وصف،',
                    'دل خرابی می‌کند یعنی اسرار را فاش می‌سازد.',
                    '.*اما صورت تمثیل:', 
                    'توضیح بیشتر در.*',
                    ' مسلمان‌گر بدانستی که بت چیست.*',
                    ' حیرت بر دو نوع است.*',
                    'این بخش جلیل،',
                    r'حضرت علی به آن پهلوان',
                    '«بطر به معنی.*',
                    '«زنان بدکار شایسته.*',
                    ' و قبل از آغاز حکایت.*',
                    'خطاب به دل گوید: ',
                    ' بس گریزند از بلا سوی بلا / بس جهند از مار سوی اژدها',
                    'بود نور نبی خورشید اعظم.*',
                    'قانع و راضی باش / بی آرام چیزی.*',
                    'جور دشمن چه کند‌گر نکشد طالب دوست / گنج و مار و گل و خار و غم و شادی به هم اند',
                    'شعرا سفتن را به کنایه.*',
                   'سید کفافی ج ۲',
                   r'[0-9]'

                                        

                    ]

def cleaning_1(text):
    """First set of cleaning and removing extra text"""

    text = normalizer.normalize(text)


    text = re.sub('ایتجا', 'اینجا', text)
    


    sub_1 = [' فرسی ', 'ایتجا', ' شخ ']
    sub_2 = ['فارسی', ' اینجا', ' شخص ']


    for patt in  remove_patterns + separators:   
        text = re.sub(patt, ' ', text)

    for i in range(len(sub_1)):
        text = re.sub(sub_1[i], sub_2[i], text)


    text =  re.sub('^ ', '', text)
    text =  re.sub(' $', '', text)
    text =  re.sub(r' */ *', ' / ', text)

    text =  re.sub(r' \. \.', '\.', text)
    text =  re.sub(' +\s', ' ', text)

    text =  re.sub(' \.$', '\.', text)
    text =  re.sub('^ *\. *', '', text)

    text =  re.sub('[۱۲۳۴۵۶۷۸۹۰]', '', text)

    
    # replace extra spaces or tabs with only one space
    text =  re.sub(r' \s+', ' ', text)


    tex =  re.sub(r'([\/\.])', r'\1', text)
    
    tex =  re.sub(r'\s+', ' ', text)

    text = re.sub('^ ', '', text)

    text = re.sub(' $', '', text)


    text = re.sub(r' \. \.', r'.', text)

    text = re.sub('و *$', '', text)
    
    # remove punc
    text = text.translate(str.maketrans('', '', punct))

    

    return text
    

all_data.loc[:, 'text'] = all_data.loc[:, 'text'].apply(lambda x:cleaning_1(x))
all_data.loc[:, 'poetry'] = all_data.loc[:, 'poetry'].apply(lambda x:cleaning_1(x))

In [ ]:
# check for very short sentences (plain text)

selected = []
for i in range(len(all_data_final['text'])):
    if len(all_data_final.loc[i, 'text'])<32:
        print(i)
        print(all_data_final.loc[i, 'text'])
        #print(all_data_not_cleaned.loc[i,'text'])
        
        selected.append(i)
print(colored(f'length of matches: {len(selected)}', 'blue'))

all_data_final.drop(index = selected, inplace=True)
all_data_final.reset_index(drop=True, inplace=True)

In [ ]:
# exporting the cleaned dataset

#all_data_final.to_csv(None, index=False)

# Augmentin the poetry

In [ ]:
# this model has been pretrained on couplets
# and will be used for augmenting poetry

bent_trained_path = None

persian_stw = stopwords_list()
aug = naw.ContextualWordEmbsAug(model_path=bent_trained_path,
                                action="substitute",
                                stopwords=persian_stw,
                                temperature=0.4, 
                                device='cuda' if tf.test.gpu_device_name() else 'cpu',
                                top_p=0.99,
                                top_k=100, 
                                aug_min=1,
                                #tokenizer =tokenizer_source,
                                aug_p=0.2
                                )

In [ ]:
# augmenting the poetry

for i in range(len(all_data)):
    
    all_data = pd.concat([all_data, 
                           pd.DataFrame({'poetry': [aug.augment(all_data.loc[i,'poetry'], n= 2)],  
                                          'text': all_data.loc[i,'text']})],
                           axis=0)
    if i % 100 == 0:
        print(i)
        with open(None, 'wb') as f:
            pickle.dump(all_data, f)

all_data.reset_index(inplace=True, drop=True)
all_data

In [ ]:
unwanted_records = []

# put records with more than one translation (augmented) into two rows

for i in range(len(all_data)):
    if type(all_data.loc[i,'poetry']) == list: 

        all_data = pd.concat([all_data, 
                             pd.DataFrame({'poetry': all_data.loc[i, 'poetry'], 
                                           'text': all_data.loc[i,'text']})],
                             axis=0)
        unwanted_records.append(i)

all_data.drop(unwanted_records, inplace=True)
all_data.reset_index(inplace=True, drop=True)

In [ ]:
# removing the records with generated unknown tokens

unk_t = []
for i in range(len(all_data)):
    if re.search('\[UNK\]',  all_data.loc[i, 'poetry']):
        print(all_data.loc[i, 'poetry'])
        unk_t.append(i)

In [ ]:
# augmenting the text
model_check_point = 'HooshvareLab/bert-fa-base-uncased'


# for text use high temperature
persian_stw = stopwords_list()
aug = naw.ContextualWordEmbsAug(model_path=model_check_point,
                                action="substitute",
                                stopwords=persian_stw,
                                temperature=0.6, 
                                device='cuda' if tf.test.gpu_device_name() else 'cpu',
                                top_p=0.8,
                                top_k=150, 
                                aug_min=3,
                                #tokenizer =tokenizer_source,
                                aug_p=0.4
                                )

In [ ]:
for i in range(len(all_data)):
    
    all_data = pd.concat([all_data, 
                           pd.DataFrame({'poetry': all_data.loc[i, 'poetry'],  
                                          'text': [aug.augment(all_data.loc[i,'text'], n= 1)]})],
                           axis=0)
    if i % 100 == 0:
        print(i)

In [ ]:
all_data.reset_index(inplace=True,drop=True)

# Cleaning other files we use

In [ ]:
# Motaradefs(synonyms) ----------------------------------

mot = pd.read_pickle(None)
df = pd.DataFrame({'word':list(mot.keys()), 'syn':list(mot.values())})

# remove stopwords

stw_ = stopwords_list()
banned_words = set(stw_)

unwanted = []
for i in range(len(df)):
    for word in banned_words:
        if df.loc[i, 'word'] == word:
            print(colored(word, 'blue'))
            print(df.loc[i, 'syn'])
            unwanted.append(i)

df.drop(unwanted, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
def clean(mot_list):
    """
    cleaning each unwanted letter or number in the words
    """
    unw = []

    for i in range(len(mot_list)):

        
        if len(mot_list[i])==0: unw.append(i)



    if len(unw)!=0:
        for index in sorted(set(unw), reverse=True):
            del mot_list[index]

        
    else :
        for i in range(len(mot_list)):
            for j in range(len(mot_list[i])):

                text = normalizer.normalize(mot_list[i][j])
                text = re.sub(r'[۱|۲|۳|۴|۵|۶|۷|۸|۹|۰|؟|!|.|،|,|?]', '', text)
                mot_list[i][j] = text 
                

    return mot_list

df.loc[:, 'syn'] = df.loc[:, 'syn'].apply(lambda x:clean(x))

In [ ]:
# removing exessive nouns with no synonyms

remove_these = []
for i in range(len(df)):
    if len(df.loc[i, 'syn']) ==0:
        print(i)
        print(df.loc[i, :])
        remove_these.append(i)

df.drop(unwanted, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
all_dict = {
    'word':[],
    'synonym':[],
    'antonym':[]
}

for i in range(len(df)):
    all_dict['word'].append(df.loc[i, 'word'])
    
    all_dict['synonym'].append(df.loc[i, 'syn'][0])
    try: all_dict['antonym'].append(df.loc[i, 'syn'][1])
    except: all_dict['antonym'].append('nothing')

all_mots = pd.DataFrame(all_dict)
all_mots.to_csv(None)


# Cleaning and augmenting the Semantic affinity dataset

In [ ]:
# main file
gherab = pd.read_pickle('.../Gherabat.pickle')

# additional files for each class
erfani = pd.read_csv('.../erfani.txt', header=None)[0].tolist()
khodae = pd.read_csv('.../khodae.txt', header=None)[0].tolist()
akhlaghi = pd.read_csv('.../akhlagh.txt', header=None)[0].tolist()
eshghi = pd.read_csv('.../love.txt', header=None)[0].tolist()

In [ ]:
import hazm
normalizer = hazm.Normalizer(persian_numbers=False)
cleaned = []


sub_1 = ['هٔ', 'أ', 'ئ', '»', 'ؤ', 'ازآن', 'هرآنچه', 'ازآنک', 'زورآور', 'وآن',
         'وآی', ' ٔ', 'ء', '؟']
sub_2 = ['ه', 'ا', 'ی', '', 'و', 'از آن', 'هر آنچه', 'از آنک', 'زور آور', 'و آن',
         'و آی', '', '', ''] # has been modified

for j in range(len(all_poems)):
    

    text = all_poems[j]
    text = normalizer.normalize(text)

    for i in range(len(sub_1)):
        text = re.sub(sub_1[i], sub_2[i], text)
    
    cleaned.append(text)

# verse into couplet
akhlaghi_b=[]
for i in range(2, len(akhlaghi), 2):
    akhlaghi_b.append(' / '.join(akhlaghi[i-2:i]))


love_b=[]
for i in range(2, len(eshghi), 2):
    love_b.append(' / '.join(eshghi[i-2:i]))

erfani_b=[]
for i in range(2, len(erfani), 2):
    erfani_b.append(' / '.join(erfani[i-2:i]))

khodae_b=[]
for i in range(2, len(khodae), 2):
    khodae_b.append(' / '.join(khodae[i-2:i]))

# all additional files in one datafile
df = pd.DataFrame({
    'topic': ['akhlaghi']*len(akhlaghi_b) + ['eshghi']*len(love_b) + ['erfan']*len(erfani_b) + ['khodavand']*len(khodae_b),
    'poetry': akhlaghi_b + love_b + erfani_b + khodae_b
})

# primary file

df_main = pd.DataFrame({
    'topic': [],
    'poetry': []
})

for k,v in gherab.items():
    df_main = pd.concat([df_main, pd.DataFrame({
    'topic': k,
    'poetry': v
})], axis=0)



# concatenating the semantica affinity files
concatenated = pd.concat([df, df_main], axis=0)
concatenated.reset_index(inplace=True, drop=True)

# removing duplicates and very short texts
concatenated.drop(concatenated[[True if len(i)<32 else False for i in concatenated['poetry']]].index, inplace=True)
concatenated.drop_duplicates(inplace=True, subset='poetry')
concatenated

In [ ]:
# augmented dataset for the semantic affinity classification
concat_path = None

concatenated.to_csv(concat_path)
concatenated = pd.read_csv(concat_path)
print('length of the augmented dataset for automatic eval: ', colored( f"{len(concatenated):,}", 'blue'))

In [ ]:
# we use the bert model trained on poetry for augmenting the poetry
# in this dataset

persian_stw = stopwords_list()
aug = naw.ContextualWordEmbsAug(model_path='mitra-mir/BERT-Persian-Poetry',
                                action="substitute",
                                stopwords=persian_stw,
                                temperature=0.3, 
                                device='cuda' if tf.test.gpu_device_name() else 'cpu',
                                top_p=0.99,
                                top_k=100, 
                                aug_min=1,
                                #tokenizer =tokenizer_source,
                                aug_p=0.2
                                )

In [ ]:
concatenated.reset_index(inplace=True, drop=True)
for i in range(len(concatenated)):
    
    concatenated = pd.concat([concatenated, 
                           pd.DataFrame({'poetry': [aug.augment(concatenated.loc[i,'poetry'], n= 1)],  
                                          'topic': concatenated.loc[i,'topic']})],
                           axis=0)
    if i % 100 == 0:
        print(i)
concatenated.reset_index(inplace=True, drop=True)

# removing augmented records with [UNK]
unk_t = []
for i in range(len(concatenated)):
    try:
        if re.search('\[UNK\]',  concatenated.loc[i, 'poetry']):
            print(concatenated.loc[i, 'poetry'])
            unk_t.append(i)

    except: print(concatenated.loc[i, 'poetry'])

concatenated = pd.read_csv('.../concatenated_augmented.csv')

### A few experiments along the way

In [ ]:
# needed data

from google.colab import drive
drive.mount('/content/drive')


hafez = pd.read_pickle('.../hafez.pickle')
molana = pd.read_pickle('.../molana.pickle')
sadi = pd.read_pickle('.../saedi.pickle')
added = pd.read_csv('.../data_added_1.csv')



hafez = pd.DataFrame([i[3:] for i in hafez], columns=['poetry', 'text'])
molana = pd.DataFrame([i[3:] for i in molana], columns=['poetry', 'text'])
sadi = pd.DataFrame([i[3:] for i in sadi], columns=['poetry', 'text'])


dfs = [hafez, sadi, molana]

In [ ]:
all_data = pd.concat([hafez, molana, sadi, added], axis=0)

In [ ]:
# exporting the cleaned dataset

#all_data_final.to_csv(None, index=False)

In [ ]:
print('removing duplication...')
molana = molana[~molana.duplicated()]
molana.reset_index(drop=True, inplace=True)

In [ ]:
all_data = all_data[~all_data.duplicated()]
all_data.reset_index(inplace=True, drop=True)

In [ ]:
added.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
for i in range(len(all_data)):
    if re.search('چه کسی می تواند بخشش بی نهایت خداوند را بشمارد', all_data.loc[i, 'text']):
        print('ff')

In [ ]:
matches = []
for i in range(len(all_data_final)):
    if re.search( '^بنابراین', all_data_final.loc[i, 'text'],):
        print(i)
        print( all_data_final.loc[i, 'text'])
        matches.append(i)
print('# of matches : ', colored(len(matches), 'blue'))

In [ ]:
# removing unwanted long sentences
selected = []
for i in range(len(all_data_final['text'])):
    if len(all_data_final.loc[i, 'text'])>295:
        print(i)
        print(all_data_final.loc[i, 'text'])
        #print(all_data_not_cleaned.loc[i,'text'])
        
        selected.append(i)
print(colored(f'length of matches: {len(selected)}', 'blue'))